# Matrix Factorization

## Khởi tạo và chuẩn hóa ma trận

In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
            
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
        # normalized data
        self.Y_data_n = self.Y_raw_data.copy()

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
            
    
    def loss(self):
        L = 0 
        for i in range(self.Y_data_n.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print ('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        if pred < 1:
            return 1 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE
        
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.values


rs = MF(Y_data, K = 2, max_iter = 1000, print_every = 1000)

rs.fit()
rs.pred(6, 1)

iter = 1000 , loss = 0.5084975490672691 , RMSE train = 0.7958110792326252


2.9379411162130262

In [2]:
print (rs.X.dot(rs.W) + rs.mu)

[[4.16624786 4.01680483 1.80731894 0.22158114 1.71830212 1.43292925
  2.37160252]
 [3.6591257  3.2865949  2.21601037 0.81566298 2.14424007 1.47944526
  2.93794112]
 [3.99858614 3.85484798 1.87324165 0.4760864  1.87747986 1.42262395
  2.46576486]
 [2.44222812 1.7004917  3.05205171 2.36267907 3.20470286 1.53737105
  4.1023503 ]
 [2.25992554 1.32036261 3.30140495 2.49024361 3.33064667 1.5921144
  4.44372716]]


# Áp dụng lên MovieLens 100k

In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.values
rate_test = ratings_test.values

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [4]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nUser-based MF, RMSE =', RMSE)

iter = 10 , loss = 5.6666634981832065 , RMSE train = 1.2110603962510282
iter = 20 , loss = 2.6423237086126456 , RMSE train = 1.0379499719985796
iter = 30 , loss = 1.344637516306602 , RMSE train = 1.029472492975513
iter = 40 , loss = 0.7534496071736536 , RMSE train = 1.0291969079150034
iter = 50 , loss = 0.48250125354877377 , RMSE train = 1.0292071482573928
iter = 60 , loss = 0.358251904331299 , RMSE train = 1.0292125917309938
iter = 70 , loss = 0.3012721655011711 , RMSE train = 1.0292139397318147
iter = 80 , loss = 0.27514178874020856 , RMSE train = 1.0292142443494858
iter = 90 , loss = 0.2631586841630353 , RMSE train = 1.0292143118994623
iter = 100 , loss = 0.2576633732885457 , RMSE train = 1.0292143267967497

User-based MF, RMSE = 1.0603799065781432


In [5]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iter = 10 , loss = 5.627300815677279 , RMSE train = 1.1759647866308531
iter = 20 , loss = 2.6193741595966102 , RMSE train = 1.0052778973877226
iter = 30 , loss = 1.3250750046123816 , RMSE train = 0.9965435064507597
iter = 40 , loss = 0.7352902392363744 , RMSE train = 0.9961867781427354
iter = 50 , loss = 0.4649860406314433 , RMSE train = 0.9961787000181426
iter = 60 , loss = 0.34103390141671786 , RMSE train = 0.9961802195401405
iter = 70 , loss = 0.28419100621754856 , RMSE train = 0.9961807357151344
iter = 80 , loss = 0.2581235473766677 , RMSE train = 0.9961808643812019
iter = 90 , loss = 0.24616935118952127 , RMSE train = 0.9961808947310101
iter = 100 , loss = 0.2406873183932131 , RMSE train = 0.9961809017643232

Item-based MF, RMSE = 1.0486424539909174


In [6]:
rs = MF(rate_train, K = 2, lam = 0, print_every = 10, learning_rate = 1, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

iter = 10 , loss = 1.0829562408750968 , RMSE train = 1.4212725301333173
iter = 20 , loss = 1.0262379415927207 , RMSE train = 1.4055257721440935
iter = 30 , loss = 0.9755228320005906 , RMSE train = 1.3905237849533638
iter = 40 , loss = 0.9299669098542266 , RMSE train = 1.376209933862853
iter = 50 , loss = 0.888871471060061 , RMSE train = 1.362594825251479
iter = 60 , loss = 0.8516538552854371 , RMSE train = 1.349657297603167
iter = 70 , loss = 0.8178248796755295 , RMSE train = 1.3373137448648758
iter = 80 , loss = 0.78697126033549 , RMSE train = 1.325564912472822
iter = 90 , loss = 0.7587417944789745 , RMSE train = 1.3143856325750538
iter = 100 , loss = 0.7328364069464384 , RMSE train = 1.3037201004671581

Item-based MF, RMSE = 1.3895662847193322


In [7]:
RMSE = rs.evaluate_RMSE(rate_train)

In [8]:
print (RMSE)

1.3037201004671581


# MovieLens 1M

In [9]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')


/var/folders/mx/__zp2dyx02g_k1_g_3n5xcrc0000gn/T/ipykernel_34684/3273828672.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_base = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')


In [13]:
ratings = ratings_base.values
ratings[:, :2] -= 1

from sklearn.model_selection import train_test_split

rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print (X_train.shape, X_test.shape)

rs = MF(rate_train, K = 2, lam = 0.1, print_every = 2, learning_rate = 2, max_iter = 10, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

ValueError: not enough values to unpack (expected 4, got 2)